This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-features/model_hot_swap).

## Model Hot Swap Tutorial

One of the biggest challenges facing organizations once they have a model trained is deploying the model:  Getting all of the resources together, MLOps configured and systems prepared to allow inferences to run.

The next biggest challenge?  Replacing the model while keeping the existing production systems running.

This tutorial demonstrates how Wallaroo model hot swap can update a pipeline step with a new model with one command.  This lets organizations keep their production systems running while changing a ML model, with the change taking only milliseconds, and any inference requests in that time are processed after the hot swap is completed.

This example and sample data comes from the Machine Learning Group's demonstration on [Credit Card Fraud detection](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud).

This tutorial provides the following:

* `ccfraud.onnx`: A pre-trained ML model used to detect potential credit card fraud.
* `xgboost_ccfraud.onnx`: A pre-trained ML model used to detect potential credit card fraud originally converted from an XGBoost model.  This will be used to swap with the `ccfraud.onnx`.
* `smoke_test.json`: A data file used to verify that the model will return a low possibility of credit card fraud.
* `high_fraud.json`: A data file used to verify that the model will return a high possibility of credit card fraud.
* Sample inference data files: Data files used for inference examples with the following number of records:
  * `cc_data_5.json`: 5 records.
  * `cc_data_1k.json`: 1,000 records.
  * `cc_data_10k.json`: 10,000 records.
  * `cc_data_40k.json`: Over 40,000 records.

## Reference

For more information about Wallaroo and related features, see the [Wallaroo Documentation Site](https://docs.wallaroo.ai).

## Steps

The following steps demonstrate the following:

* Connect to a Wallaroo instance.
* Create a workspace and pipeline.
* Upload both models to the workspace.
* Deploy the pipe with the `ccfraud.onnx` model as a pipeline step.
* Perform sample inferences.
* Hot swap and replace the existing model with the `xgboost_ccfraud.onnx` while keeping the pipeline deployed.
* Conduct additional inferences to demonstrate the model hot swap was successful.
* Undeploy the pipeline and return the resources back to the Wallaroo instance.

### Load the Libraries

Load the Python libraries used to connect and interact with the Wallaroo instance.

In [9]:
import wallaroo
from wallaroo.object import EntityNotFoundError

import pandas as pd
import os
os.environ["ARROW_ENABLED"]="True"
if "ARROW_ENABLED" in os.environ:
    arrowEnabled = os.environ["ARROW_ENABLED"]
else:
    arrowEnabled = False
    
# used to display dataframe information without truncating
pd.set_option('display.max_colwidth', None)

### Open a Connection to Wallaroo

The first step is to connect to Wallaroo through the Wallaroo client.

This is accomplished using the `wallaroo.Client(api_endpoint, auth_endpoint, auth_type command)` command that connects to the Wallaroo instance services.

The `Client` method takes the following parameters:

* **api_endpoint** (*String*): The URL to the Wallaroo instance API service.
* **auth_endpoint** (*String*): The URL to the Wallaroo instance Keycloak service.
* **auth_type command** (*String*): The authorization type.  In this case, `SSO`.

The URLs are based on the Wallaroo Prefix and Wallaroo Suffix for the Wallaroo instance.  For more information, see the [DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/).  In the example below, replace "YOUR PREFIX" and "YOUR SUFFIX" with the Wallaroo Prefix and Suffix, respectively.

If connecting from within the Wallaroo instance's JupyterHub service, then only `wl = wallaroo.Client()` is required.

Once run, the `wallaroo.Client` command provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Depending on the configuration of the Wallaroo instance, the user will either be presented with a login request to the Wallaroo instance or be authenticated through a broker such as Google, Github, etc.  To use the broker, select it from the list under the username/password login forms.  For more information on Wallaroo authentication configurations, see the [Wallaroo Authentication Configuration Guides](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-sso-authentication/).

In [10]:
# Remote Login

# wallarooPrefix = "YOUR PREFIX"
# wallarooSuffix = "YOUR SUFFIX"

# wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
#                     auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
#                     auth_type="sso")

# Internal Login

wl = wallaroo.Client()

### Set the Variables

The following variables are used in the later steps for creating the workspace, pipeline, and uploading the models.  Modify them according to your organization's requirements.

In [11]:
workspace_name = 'hotswapworkspace'
pipeline_name = 'hotswappipeline'
original_model_name = 'ccfraudoriginal'
original_model_file_name = './ccfraud.onnx'
replacement_model_name = 'ccfraudreplacement'
replacement_model_file_name = './xgboost_ccfraud.onnx'

In [12]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(pipeline_name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(pipeline_name)
    return pipeline

### Create the Workspace

We will create a workspace based on the variable names set above, and set the new workspace as the `current` workspace.  This workspace is where new pipelines will be created in and store uploaded models for this session.

Once set, the pipeline will be created.

In [13]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)
pipeline

name,hotswappipeline
created,2023-02-10 23:18:49.896972+00:00
last_updated,2023-02-10 23:18:49.896972+00:00
deployed,(none)
tags,
versions,24732338-1073-4a7c-a394-44f99c439883
steps,


### Upload Models

We can now upload both of the models.  In a later step, only one model will be added as a [pipeline step](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipeline/#add-a-step-to-a-pipeline), where the pipeline will submit inference requests to the pipeline.

In [14]:
original_model = wl.upload_model(original_model_name , original_model_file_name)
replacement_model = wl.upload_model(replacement_model_name , replacement_model_file_name)

In [28]:
wl.list_models()

Name,# of Versions,Owner ID,Last Updated,Created At
ccfraudreplacement,1,"""""",2023-02-10 23:18:51.854121+00:00,2023-02-10 23:18:51.854121+00:00
ccfraudoriginal,1,"""""",2023-02-10 23:18:51.705621+00:00,2023-02-10 23:18:51.705621+00:00


### Add Model to Pipeline Step

With the models uploaded, we will add the original model as a pipeline step, then deploy the pipeline so it is available for performing inferences.

In [15]:
pipeline.add_model_step(original_model)
pipeline

name,hotswappipeline
created,2023-02-10 23:18:49.896972+00:00
last_updated,2023-02-10 23:18:49.896972+00:00
deployed,(none)
tags,
versions,24732338-1073-4a7c-a394-44f99c439883
steps,


In [16]:
pipeline.deploy()

Waiting for deployment - this will take up to 45s .............. ok


name,hotswappipeline
created,2023-02-10 23:18:49.896972+00:00
last_updated,2023-02-10 23:18:56.308649+00:00
deployed,True
tags,
versions,"2311acac-0701-4a2c-bfc3-ab28311ae9f0, 24732338-1073-4a7c-a394-44f99c439883"
steps,ccfraudoriginal


In [29]:
pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.48.2.94',
   'name': 'engine-7968cdb974-6msmq',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'hotswappipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'ccfraudoriginal',
      'version': '12e7ad13-3ff3-45b6-b416-43eef5f2ef64',
      'sha': 'bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.48.2.95',
   'name': 'engine-lb-74b4969486-rhm9g',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

### Verify the Model

The pipeline is deployed with our model.  The following will verify that the model is operating correctly.  The `high_fraud.json` file contains data that the model should process as a high likelihood of being a fraudulent transaction.

In [22]:
if arrowEnabled:
    result = pipeline.infer_from_file('./data/high_fraud.df.json')
else:
    result = pipeline.infer_from_file('./data/high_fraud.json')
display(result)

,time,in.tensor,out.dense_1,check_failures
0,2023-02-10 23:27:20.640,"[1.0678324729, 18.1555563975, -1.6589551058, 5.2111788045, 2.3452470645, 10.4670835778, 5.0925820522, 12.8295153637, 4.9536770468, 2.3934736228, 23.912131818, 1.759956831, 0.8561037518, 1.1656456469, 0.5395988814, 0.7784221343, 6.7580610727, 3.9274118477, 12.4621782767, 12.3075382165, 13.7879519066, 1.4588397512, 3.6818346868, 1.753914366, 8.4843550037, 14.6454097667, 26.8523774363, 2.7165292377, 3.0611957069]",[0.981199],0


### Replace the Model

The pipeline is currently deployed and is able to handle inferences.  The model will now be replaced without having to undeploy the pipeline.  This is done using the pipeline method [`replace_with_model_step(index, model)`](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-reference-guide/pipeline/#Pipeline.replace_with_model_step).  Steps start at `0`, so the method called below will replace step 0 in our pipeline with the replacement model.

As an exercise, this deployment can be performed while inferences are actively being submitted to the pipeline to show how quickly the swap takes place.

In [23]:
pipeline.replace_with_model_step(0, replacement_model).deploy()

 ok


name,hotswappipeline
created,2023-02-10 23:18:49.896972+00:00
last_updated,2023-02-10 23:27:21.934954+00:00
deployed,True
tags,
versions,"11b2627c-a621-4367-8a53-db297dc52a32, 2311acac-0701-4a2c-bfc3-ab28311ae9f0, 24732338-1073-4a7c-a394-44f99c439883"
steps,ccfraudoriginal


In [30]:
# Display the pipeline
pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.48.2.94',
   'name': 'engine-7968cdb974-6msmq',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'hotswappipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'ccfraudoriginal',
      'version': '12e7ad13-3ff3-45b6-b416-43eef5f2ef64',
      'sha': 'bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.48.2.95',
   'name': 'engine-lb-74b4969486-rhm9g',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

### Verify the Swap

To verify the swap, we'll submit a set of inferences to the pipeline using the new model.  We'll display just the first 5 rows for space reasons.

In [20]:
if arrowEnabled:
    result = pipeline.infer_from_file('./data/cc_data_1k.df.json')
    display(result.loc[0:4,:])
else:
    result = pipeline.infer_from_file('./data/cc_data_1k.json')
    display(result)

,time,in.tensor,out.dense_1,check_failures
0,2023-02-10 23:23:10.539,"[-1.0603297501, 2.3544967095, -3.5638788326, 5.1387348926, -1.2308457019, -0.7687824608, -3.5881228109, 1.8880837663, -3.2789674274, -3.9563254554, 4.0993439118, -5.6539176395, -0.8775733373, -9.131571192, -0.6093537873, -3.7480276773, -5.0309125017, -0.8748149526, 1.9870535692, 0.7005485718, 0.9204422758, -0.1041491809, 0.3229564351, -0.7418141657, 0.0384120159, 1.0993439146, 1.2603409756, -0.1466244739, -1.4463212439]",[0.99300325],0
1,2023-02-10 23:23:10.539,"[-1.0603297501, 2.3544967095, -3.5638788326, 5.1387348926, -1.2308457019, -0.7687824608, -3.5881228109, 1.8880837663, -3.2789674274, -3.9563254554, 4.0993439118, -5.6539176395, -0.8775733373, -9.131571192, -0.6093537873, -3.7480276773, -5.0309125017, -0.8748149526, 1.9870535692, 0.7005485718, 0.9204422758, -0.1041491809, 0.3229564351, -0.7418141657, 0.0384120159, 1.0993439146, 1.2603409756, -0.1466244739, -1.4463212439]",[0.99300325],0
2,2023-02-10 23:23:10.539,"[-1.0603297501, 2.3544967095, -3.5638788326, 5.1387348926, -1.2308457019, -0.7687824608, -3.5881228109, 1.8880837663, -3.2789674274, -3.9563254554, 4.0993439118, -5.6539176395, -0.8775733373, -9.131571192, -0.6093537873, -3.7480276773, -5.0309125017, -0.8748149526, 1.9870535692, 0.7005485718, 0.9204422758, -0.1041491809, 0.3229564351, -0.7418141657, 0.0384120159, 1.0993439146, 1.2603409756, -0.1466244739, -1.4463212439]",[0.99300325],0
3,2023-02-10 23:23:10.539,"[-1.0603297501, 2.3544967095, -3.5638788326, 5.1387348926, -1.2308457019, -0.7687824608, -3.5881228109, 1.8880837663, -3.2789674274, -3.9563254554, 4.0993439118, -5.6539176395, -0.8775733373, -9.131571192, -0.6093537873, -3.7480276773, -5.0309125017, -0.8748149526, 1.9870535692, 0.7005485718, 0.9204422758, -0.1041491809, 0.3229564351, -0.7418141657, 0.0384120159, 1.0993439146, 1.2603409756, -0.1466244739, -1.4463212439]",[0.99300325],0
4,2023-02-10 23:23:10.539,"[0.5817662108, 0.097881551, 0.1546819424, 0.4754101949, -0.1978862306, -0.4504344854, 0.0166540447, -0.0256070551, 0.0920561602, -0.2783917153, 0.0593299441, -0.0196585416, -0.4225083157, -0.1217538877, 1.5473094894, 0.2391622864, 0.3553974881, -0.7685165301, -0.7000849355, -0.1190043285, -0.3450517133, -1.1065114108, 0.2523411195, 0.0209441826, 0.2199267436, 0.2540689265, -0.0450225094, 0.1086773898, 0.2547179311]",[0.0010916889],0


### Undeploy the Pipeline

With the tutorial complete, the pipeline is undeployed to return the resources back to the Wallaroo instance.

In [ ]:
pipeline.undeploy()

Waiting for undeployment - this will take up to 45s ..................